# On The Free Energy Penalty of Cavity Formation In Salt Solutions: Rethinking the Terms “Kosmotropic” and “Chaotropic”.

Stefan Hervø-Hansen<sup>a,*</sup> and Nobuyuki Matubayasi<sup>a,*</sup>.<br><br>
<sup>a</sup> Division of Chemical Engineering, Graduate School of Engineering Science, Osaka University, Toyonaka, Osaka 560-8531, Japan.<br>
<sup>*</sup> To whom correspondence may be addressed: stefan@cheng.es.osaka-u.ac.jp and nobuyuki@cheng.es.osaka-u.ac.jp.

## Simulations

### Introduction

### Methods & Materials

### References



## Import of Python Modules & Auxiliary Functions

In [ ]:
# Notebook dependent libs
import numpy as np
import matplotlib.pyplot as plt
import os, time
from shutil import copyfile
from distutils.spawn import find_executable
import parmed as pmd
import mdtraj as md

# Check for external programs
if None in [find_executable('packmol')]:
    print('WARNING: External program missing!')

# homedir = !pwd
# homedir = homedir[0]

homedir = '/home/lin/Cavity-Formation'
mddir = '/home/lin/Cavity-Formation/md'
print(homedir)

## Molecular Dynamics Simulations
### Simulation settings

In [ ]:
# Write this code
salts = {
    'NaF': {'cation': 'Na', 'anion': 'F'},
    'NaCl': {'cation': 'Na', 'anion': 'Cl'},
    'NaI': {'cation': 'Na', 'anion': 'I'}
}

concentrations = {
#     0.0:{'N_water':10000,'N_cation':0,'N_anion':0},
    0.25:{'N_water':10000,'N_cation':45,'N_anion':45},
#     0.5:{'N_water':10000,'N_cation':90,'N_anion':90},
#     0.75:{'N_water':10000,'N_cation':135,'N_anion':135},
#     1.00:{'N_water':10000,'N_cation':180,'N_anion':180}
}

#### Generation of initial configuration using Packmol

In [ ]:
packmol_script="""
tolerance 2.0
filetype pdb
output system.pdb

add_box_sides 1.0

structure ../../../../PDB_files/OPC.pdb
        number {N_water}
        inside cube 0. 0. 0. 63.
end structure

structure ../../../../PDB_files/Ions/{anion}.pdb
        number {N_anion}
        inside cube 0. 0. 0. 63.
end structure

structure ../../../../PDB_files/Ions/{cation}.pdb
        number {N_cation}
        inside cube 0. 0. 0. 63.
end structure
"""

topology_script = """
; topol.top
[ defaults ]
; nbfunc comb-rule gen-pairs fudgeLJ fudgeQQ
1 2 yes 0.5 0.8333

#include "../../../Force_fields/OPC.itp"
#include "../../../Force_fields/{cation}.itp"
#include "../../../Force_fields/{anion}.itp"

[ system ]
{cation}{anion} in water

[ molecules ]
; Compound nmols
HOH 10000
{cation} {N_cation}
{anion} {N_anion}
"""

In [ ]:
# for concentration,N in concentrations.items():
#     print(N['N_cation'])

%cd /home/lin/Cavity-Formation/Simulations

homedir = '/home/lin'
mddir = '/home/lin/Cavity-Formation/md'
outputdir = '/home/lin/Cavity-Formation/Simulations'

for salt,S in salts.items():
    for concentration,N in concentrations.items():
        making directory
        %mkdir {mddir}/{salt}
        %mkdir {outputdir}/{salt}
        %mkdir {mddir}/{salt}/{concentration} 
        %mkdir {outputdir}/{salt}/{concentration}
        folder_names = ['sys','em','nvt','npt']
        working_dirs = {} 
        for folder_name in folder_names:
            %mkdir {mddir}/{salt}/{concentration}/{folder_name}
#             working_dirs_key = f'{mddir}/{salt}/{concentration}/{folder_name}'
#             working_dirs[f'{salt}_{concentration}'] = f'{mddir}/{salt}/{concentration}/{folder_name}' ＃上書きされるので後で修正
        
        %cd {mddir}/{salt}/{concentration}/sys
        # Generate initial configuration using packmol
        with open('packmol.inp', 'w') as f:
            f.write(packmol_script.format(homedir=homedir,cation=S['cation'], anion=S['anion'],N_water=N['N_water'],
                                          N_anion=N['N_anion'],N_cation=N['N_cation']))
        f.close()
        !packmol < packmol.inp

        # Convert packmol pdb to gromacs format
        init = pmd.load_file('system.pdb')
        init.save('system.gro')

        # Write gromacs topology
        with open('topol.top', 'w') as f:
            f.write(topology_script.format(cation=S['cation'], anion=S['anion'],N_water=N['N_water'],
                                          N_anion=N['N_anion'],N_cation=N['N_cation']))
        f.close()
        print(working_dirs)
        print('Wrote initial configurations to'+os.getcwd())

#### Submit script

In [ ]:
md_fist_script = """
#!/bin/bash
#PBS -l nodes=nu-g01:ppn=36

if [ "$PBS_O_WORKDIR" != "" ];then
  cd $PBS_O_WORKDIR
fi
NUMNODES=`wc -l < $PBS_NODEFILE`

source /data2/APPLI/packages/gromacs/gromacs-2016.5-intel-double.rc
"""

md_script_unit = """

#em

 cd em

gmx_mpi_d grompp -f ../../../MDP_files/em.mdp -c ../sys/system.gro -p ../sys/topol.top -o solution_min.tpr -maxwarn 1
mpirun -machinefile $PBS_NODEFILE -np $NUMNODES gmx_mpi_d mdrun -v -deffnm solution_min

cd ..

#NVT

cd nvt

gmx_mpi_d grompp -f ../../../MDP_files/nvt.mdp -c ../em/solution_min.gro -p ../sys/topol.top -o solution_nvt.tpr
mpirun -machinefile $PBS_NODEFILE -np $NUMNODES gmx_mpi_d mdrun -v -deffnm solution_nvt

cd ..

#NPT

cd npt

gmx_mpi_d grompp -f ../../../MDP_files/npt.mdp -c ../nvt/solution_nvt.gro -p ../sys/topol.top -o solution_npt.tpr
mpirun -machinefile $PBS_NODEFILE -np $NUMNODES gmx_mpi_d mdrun -v -deffnm solution_npt

cd ../

#md

cd md

gmx_mpi_d grompp -f ../../../MDP_files/md.mdp -c ../npt/solution_npt.gro -p ../sys/topol.top -o solution_run.tpr
mpirun -machinefile $PBS_NODEFILE -np $NUMNODES gmx_mpi_d mdrun -v -deffnm solution_run 

cd ../..
"""

# RESEARCH NOTES
_will be deleted when the study is finished_